In [1]:
from kafka import KafkaProducer, KafkaConsumer
from kafka import KafkaClient
import os
import io
import random
import avro.schema
from avro.io import DatumWriter
import json
import uuid
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.avro.functions import from_avro, to_avro

##os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.apache.kafka:kafka-clients:2.8.1'

findspark.init('/home/camilli/.local/lib/python3.8/site-packages/pyspark')

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

topic = 'operacoes'
topic_enriquecido = 'test2'
bootstrapServers = 'localhost:9092'

# `from_avro` requires Avro schema in JSON string format.
jsonFormatSchema = open("operacoes.avsc", "r").read()

# .config('spark.jars.packages', 'spark-sql-kafka-0-10_2.13,spark-avro_2.13')\

22/07/03 22:05:06 WARN Utils: Your hostname, BlackBerry resolves to a loopback address: 127.0.1.1; using 172.25.139.188 instead (on interface eth0)
22/07/03 22:05:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/03 22:05:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


PySpark Version :3.1.2
PySpark Version :3.1.2


In [36]:
import mysql.connector
import pandas as pd

df = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", bootstrapServers)\
  .option("subscribe", topic)\
  .option("startingOffsets", "earliest") \
  .load()

#df.selectExpr("CAST(value AS STRING) as json")
# output = df\
#   .select(from_avro("value", jsonFormatSchema).alias("operations"))

# query = df \
#     .writeStream \
#     .format("csv") \
#     .option("path","/home/camilli/projetos/checkpoint") \
#     .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
#     .start()

# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
#             .writeStream \
#             .format("kafka") \
#             .option("kafka.bootstrap.servers", bootstrapServers) \
#             .option("subscribe", topic)\
#             .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
#             .start()

# df.selectExpr("CAST(value AS STRING)", "CAST(timestamp AS TIMESTAMP)") \
df1 = df.select(from_avro("value", jsonFormatSchema).alias("operations"))


In [35]:
conn = mysql.connector.connect(database='spark',
                               host="localhost",
                               user='root',
                               password='root')

db_cursor = conn.cursor()
select = 'Select idClient, name, birth, accManager, Account, score from enrichment'
db_cursor.execute(select)
rows = db_cursor.fetchall()
df_cli = pd.DataFrame(rows)
conn.close()

df2 = spark.createDataFrame(df_cli)

df2.show()

+---+------------+----------+-------------+------+---+
|  0|           1|         2|            3|     4|  5|
+---+------------+----------+-------------+------+---+
|  1|   Ana Souza|1985-01-03| Daniel Lemos|558267|910|
|  2|Felipe Silva|1993-10-03|Marcelo Lemos|134862|450|
+---+------------+----------+-------------+------+---+



In [13]:
        # .select("data.*", "timestamp")

# df1.writeStream \
# .format("console") \
# .outputMode("append") \
# .option("truncate","false")\
# .start()

df1.writeStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", bootstrapServers) \
            .option("subscribe", topic_enriquecido)\
            .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
            .start()

##df.show()

22/07/18 01:47:55 ERROR MicroBatchExecution: Query [id = 3b203559-5d75-482b-8fc7-d9e5dce60b54, runId = f8f10c30-de9f-4b9d-8966-0d8940e86178] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:645)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.$anonfun$applyOrElse$4(MicroBatchExecution.scala:104)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:97)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:82)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDown$1(TreeNode.scala:318)
	at org.